In [79]:
import json as js
# import data
def replacing(i):
    i = i.encode('ascii', 'ignore')
    i = i.replace('&', '').replace('(', '').replace(')','')
    i = i.replace('\'', '').replace('\\', '').replace(',','')
    i = i.replace('.', '').replace('%', '').replace('/','')
    i = i.replace('"', '')    
    return str.lower(''.join([c for c in i if not c.isdigit()]))

with open('./cooking/train.json') as json_data:
    data = js.load(json_data)
    json_data.close()
train_labels = [item['cuisine'].encode('ascii', 'ignore') for item in data]
ingredients = [item['ingredients'] for item in data]
unique_cuisines = set(classes)
unique_ingredients = set()
for ingredient in ingredients:
    for item in ingredient:
        unique_ingredients.add(replacing(item))
print 'There are %d samples in the training set' % len(classes)
print 'There are %d types of cuisine' % len(unique_cuisines)
print 'There are %d types of unique ingredients' % len(unique_ingredients)

There are 39774 samples in the training set
There are 20 types of cuisine
There are 6697 types of unique ingredients


In [80]:
# convert data to matrix
train_data = []
unique_ingredients = list(unique_ingredients)
for item in data:
    current_ingredients = [replacing(ingredient) for ingredient in item['ingredients']]
    feature = []
    for ingredient in unique_ingredients:
        if ingredient in current_ingredients:
            feature.append(1)
        else:
            feature.append(0)
    train_data.append(feature)
train_data = np.array(train_data)
train_data.shape

(39774, 6697)

In [82]:
# Using Naive Bayes Classifier to perfrom 3-fold cross-validation
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.model_selection import cross_val_score
clf1 = GaussianNB()
clf2 = BernoulliNB()
scores1 = cross_val_score(clf1, train_data, train_labels, cv=3)
scores2 = cross_val_score(clf2, train_data, train_labels, cv=3)
#result = [(ref == res, ref, res) for (ref, res) in zip(train_labels, clf2.predict(train_data))]
#accuracy_learn = sum (r[0] for r in result) * 1.0 / len(result)
print "3-Fold Cross Validation results of GaussianNB: ",
print scores1
print "Average Accuracy is: %.4f" % scores1.mean()
print "3-Fold Cross Validation results of BernoulliNB: ",
print scores2
print "Average Accuracy is: %.4f" % scores2.mean()

3-Fold Cross Validation results of GaussianNB:  [ 0.37909975  0.38192926  0.377377  ]
Average Accuracy is: 0.3795
3-Fold Cross Validation results of BernoulliNB:  [ 0.68348036  0.68247983  0.68600966]
Average Accuracy is: 0.6840


In [83]:
# Using Logistic Regression Classifier to perform 3-fold cross-validation
from sklearn.linear_model import LogisticRegression
clf3 = LogisticRegression()
scores3 = cross_val_score(clf3, train_data, train_labels, cv=3)
print "3-Fold Cross Validation results of Logistic Regression: ",
print scores3
print "Average Accuracy is: %.4f" % scores3.mean()

3-Fold Cross Validation results of Logistic Regression:  [ 0.7749378   0.77366317  0.7789013 ]
Average Accuracy is: 0.7758


In [ ]:
# Using Logistic Regression Classifier to train all the data and generating labels for test data
import csv
with open('./cooking/test.json') as json_data:
    data = js.load(json_data)
    json_data.close()
ingredients = [item['ingredients'] for item in data]
test_data = []
for item in data:
    current_ingredients = [replacing(ingredient) for ingredient in item['ingredients']]
    feature = []
    for ingredient in unique_ingredients:
        if ingredient in current_ingredients:
            feature.append(1)
        else:
            feature.append(0)
    test_data.append(feature)
test_data = np.array(test_data)
clf3.fit(train_data, train_labels)
results = clf3.predict(test_data)
with open('result.csv', 'wb') as outcsv:
    writer = csv.writer(outcsv)
    writer.writerow(['id', 'cuisine'])
    for i in range(len(results)):
        writer.writerow([data[i]['id'], results[i]])